In [3]:
# imports
import pandas as pd
from datetime import datetime
import finnhub
import numpy as np
import matplotlib.pyplot as plot
from sklearn.preprocessing import MinMaxScaler

In [4]:
client = finnhub.Client(api_key='bua9lb748v6q418gd0i0')

def get_candlestick_data(ticker, timeframe, start, end):
    # I think I need to add some await functionality to the finnhub API call
    data = client.stock_candles(ticker, timeframe, start, end)
    del data['s']
    df = pd.DataFrame.from_dict(data)
    df['t'] = df['t'].apply(lambda x: datetime.fromtimestamp(x))
    df = df.rename(columns={'c': 'Close', 'h': 'High', 'l': 'Low', 'o': 'Open', 't': 'Date', 'v': 'Volume'})
    df.set_index('Date', inplace=True)
    df = df[['Open', 'High', 'Low', 'Close', 'Volume']]
    return df

In [5]:
amazon_data = get_candlestick_data('AMZN', 'D', 1281764208, 1603859231)
print(amazon_data)

                            Open         High          Low        Close  \
Date                                                                      
2010-08-15 17:00:00   123.610001   127.370003   123.040001   126.070000   
2010-08-16 17:00:00   127.339996   129.979996   126.790001   128.860001   
2010-08-17 17:00:00   129.250000   130.809998   128.259995   129.649994   
2010-08-18 17:00:00   129.220001   130.020004   126.820000   127.570000   
2010-08-19 17:00:00   127.199997   128.039993   126.019997   127.760002   
...                          ...          ...          ...          ...   
2020-10-20 17:00:00  3210.129883  3232.239990  3160.000000  3184.939941   
2020-10-21 17:00:00  3189.870117  3198.000000  3122.250000  3176.399902   
2020-10-22 17:00:00  3191.000000  3205.330078  3140.000000  3204.399902   
2020-10-25 17:00:00  3196.020020  3281.419922  3153.540039  3207.040039   
2020-10-26 17:00:00  3223.000000  3291.179932  3212.260010  3286.330078   

                      Vo

In [6]:
open_data = amazon_data.iloc[:, 0:5].values
open_data = open_data.reshape(-1,5)

scaler = MinMaxScaler(feature_range = (0, 1))

open_data = scaler.fit_transform(open_data)
open_data

array([[2.21953517e-04, 5.63206103e-04, 2.34809739e-04, 6.69080487e-04,
        1.35267977e-01],
       [1.31127371e-03, 1.32484535e-03, 1.34941164e-03, 1.48782442e-03,
        1.79012437e-01],
       [1.86907744e-03, 1.56705365e-03, 1.78633368e-03, 1.71965310e-03,
        3.01855515e-01],
       ...,
       [8.96032592e-01, 8.98763012e-01, 8.96957304e-01, 9.04025034e-01,
        1.11152026e-01],
       [8.97498655e-01, 9.20967291e-01, 9.00981771e-01, 9.04799799e-01,
        2.15817006e-01],
       [9.05377977e-01, 9.23815424e-01, 9.18434942e-01, 9.28067977e-01,
        1.46593759e-01]])

In [7]:
open_data.shape # (2570, 5)
data_train = open_data[:2056]
data_test = open_data[2030:]

In [8]:
# Create x_train and y_train tensors
x_train = []
y_train = []

# ! what about stocks that don't have at least a year's worth of trading data
# 2056 - 7 = 2049
for i in range(253, 2049):
    x_train.append(data_train[i-253:i])
    y_train.append(data_train[i:i+7])

x_train = np.asarray(x_train)
y_train = np.asarray(y_train)

# x_train.shape = (1796, 253, 5)
# y_train.shape = (1796, 7, 5)

In [9]:
# Create x_test and y_test tensors
x_test = []
y_test = []

# data_test.shape = (540, 5)
for i in range(253, 533):
    x_test.append(data_test[i-253:i])
    y_test.append(data_test[i:i+7])

x_test = np.asarray(x_test)
y_test = np.asarray(y_test)

# x_test.shape = (280, 253, 5)
# y_test.shape = (280, 7, 5)

In [10]:
y_train.shape[1]

7

In [38]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras.callbacks import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout

model = Sequential([
  layers.LSTM(units=96, return_sequences=False, input_shape=(253, 5)),
  layers.RepeatVector(7),
  layers.Dropout(0.2),
  layers.LSTM(units=96, return_sequences=True),
  layers.Dropout(0.2),
  layers.LSTM(units=96, return_sequences=True),
  layers.Dropout(0.2),
  layers.LSTM(units=96, return_sequences=True),
  layers.Dropout(0.2),
  layers.LSTM(units=96, return_sequences=True),
  layers.Dropout(0.2),
  layers.LSTM(units=96, return_sequences=True),
  layers.Dropout(0.2),
  layers.TimeDistributed(Dense(5))
])

In [39]:
model.compile(loss="mean_squared_error", optimizer="adam")

mcp_save = ModelCheckpoint('mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')

model.fit(x_train,y_train, epochs=1, batch_size=32, validation_split=0.15, callbacks=[mcp_save,earlyStopping])

48/48 [==============================] - 12s 134ms/step - loss: 0.0103 - val_loss: 0.0484
